In [8]:
# assistant: 저는 카카오 서비스 챗봇입니다.
# You:반갑습니다.
# AI assistant: 안녕하세요!반갑습니다. 도움이 필요하신 것이 있나요?
# 처음 화면에 뜨도록 하기 , 수업 시간에 프롬프트, 백터DB, (function call)을 사용하도록.

#on_sand 프롬프트 만들고, 백터 DB만들고, 유저 내용은 function call 로 ....


In [1]:
import json
import openai
import tkinter as tk
import pandas as pd
from tkinter import scrolledtext
import tkinter.filedialog as filedialog
openai.api_key = ''

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import os
import pickle


In [3]:
with open("/Users/kakao/vs_code/AI_LLM/AI_LLM/project_data_카카오톡채널.txt", "r") as f:
    corpus = []
    for line in f:
        df = f.readlines()
        corpus.append(df)

In [5]:
import chromadb

client = chromadb.PersistentClient()

collection = client.get_or_create_collection(
    name="kakao_chatbot",
    #metadata={"hnsw:space": "cosine"}# l2 is the default, metadata:유사도를 무엇으로 만드는지 설정하는 것, l2가 디폴트
)


In [15]:
# 데이터 인덱스
ids = []
# 메타데이터
doc_meta = []
# 벡터로 변환 저장할 텍스트 데이터로 ChromaDB에 Embedding 데이터가 없으면 자동으로 벡터로 변환해서 저장
documents = []

for idx, text in enumerate(corpus):
    id = f"#{idx + 1}"  # #가 추가된 형태로 id 생성

    document = f"Document {idx + 1}:\n#{text}"
    meta = {
        "index": idx + 1  # 예시로 추가된 메타데이터 필드
    }

    ids.append(id)
    doc_meta.append(meta)
    documents.append(document)


In [17]:
documents

['Document 1:\n#[\'\\n\', \'#이해하기\\n\', \'이 문서는 카카오톡 채널 API를 소개합니다.\\n\', \'\\n\', \'#기능 소개\\n\', \'카카오톡 채널(구:플러스친구)은 카카오톡 사용자들에게 다양한 서비스 소식을 메시지와 게시물 형태로 전파할 수 있는 서비스입니다. 카카오톡 채널은 친구인 사용자들에게 마케팅(Marketing) 메시지를 보내는 기능을 제공합니다. 친구란 카카오톡 채널을 친구로 추가한 사용자를 말합니다. 카카오톡 채널 메시지는 비용 절감을 위해 사용자의 성별, 나이, 지역, 등급 등 정보를 토대로 친구 그룹을 만들어서 보다 높은 효과가 기대되는 사용자들에게만 발송하는 것도 가능합니다.\\n\', \'카카오톡 채널을 활용하여 서비스와 사용자의 관계를 더욱 긴밀하게 유지할 수 있습니다. 예를 들면 카카오톡 채널 메시지를 통해 사용자에게 서비스 웹 페이지 방문을 유도하거나 유익한 상품 정보의 링크를 제공하는 것이 가능합니다. 1:1 채팅, 스마트채팅, 봇 등 유용한 추가 기능들도 이용할 수 있습니다.\\n\', \'\\n\', \'카카오톡 채널 API는 크게 두 가지의 기능을 제공합니다. 카카오톡 사용자를 위한 카카오톡 채널 추가 및 채팅 API, 다른 카카오톡 채널 관리자가 보다 편리하게 고객 그룹을 관리할 수 있도록 도와주는 카카오톡 채널 고객 관리 API가 있습니다. 두 API 모두 카카오톡 채널 프로필 ID를 사용해 요청하지만, 각각 역할과 제공 방식이 다릅니다.\\n\', \'\\n\', "카카오톡 채널 API를 사용하려면 앱과 카카오톡 채널이 연결되어 있어야 합니다. 또한 사용자의 \'카카오톡 채널 추가 상태\' 제공 동의가 필요합니다. 자세한 안내 및 설정 방법은 설정하기를 참고합니다.\\n", \'\\n\', \'참고: 카카오톡 채널 프로필 ID 확인 방법\\n\', \'[카카오톡 채널 관리자센터] > [관리] > [상세설정]에서 카카오톡 채널의 채널 URL을 확인할 수 있습니다. 채널 URL에서

In [5]:
# 데이터 인덱스
ids = []
# 메타데이터
doc_meta = []
# 벡터로 변환 저장할 텍스트 데이터로 ChromaDB에 Embedding 데이터가 없으면 자동으로 벡터로 변환해서 저장
documents = []

for idx in range(len(filter_df)):
    item = filter_df.iloc[idx]
    id = item['Name'].lower().replace(' ','-')

    document = f"{item['Name']}: {item['Synopsis']} : {str(item['Cast']).strip().lower()} : {str(item['Genre']).strip().lower()}"
    meta = {
        "rating" : item['Rating']
    }

    ids.append(id)
    doc_meta.append(meta)
    documents.append(document)

# DB 저장
collection.add(
    documents=documents,
    metadatas=doc_meta,
    ids=ids
)
# # DB 쿼리
# collection.query(
#     query_texts=["romantic comedy drama"],
#     n_results=5,
# )

NameError: name 'filter_df' is not defined

In [4]:
response = collection.query(
   query_texts=["comedy drama"],
    n_results=10,
)

NameError: name 'collection' is not defined

In [2]:
def recommend_Drama(query:str, temperature=1,max_tokens=1024):
    
  response = collection.query(
      query_texts = [query],
      n_results=10,
  )

  srchres = [];
  for v in response['documents'][0]:
    item = v.split(':')
    srchres.append({
        "title" : item[0].strip(),
        "cast" : item[2].strip(),
        "genre" : item[3].strip(),
    })

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "assistant", "content": f"srchres"},
            {"role": "user", "content": query},
            {"role": "system", "content": "제목, 출연진, 장르를 구분해서 알려줘. 마크다운 형식으로 알려줘."},

        ],
        temperature=temperature,
        max_tokens=max_tokens,
    )

    return completion.choices[0].message.content

In [3]:
res = recommend_Drama("한국 코미디 드라마 10개 추천 해 줘.")

print(res)

NameError: name 'collection' is not defined